In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
kgs.cache_dir_read = kgs.cache_dir_write

/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
kgs.disable_caching = True

too low
only doing style A in test mode
styleB not active yet
False False
(4901, 4901)


In [3]:
data = kgs.load_all_train_data(validation_only=True)[::25]
model.models[0].batch_size = 1
model.models[1].run_in_parallel = True
data_out = model.infer(data)
model.models[1].run_in_parallel = False

100%|██████████| 21/21 [00:01<00:00, 15.13it/s]


Clearing pytorch


Processing in parallel:   0%|          | 0/20 [00:00<?, ?it/s]

CUDA_VISIBLE_DEVICES= 5
CUDA_VISIBLE_DEVICES= 3
CUDA_VISIBLE_DEVICES= 6
CUDA_VISIBLE_DEVICES= 1
CUDA_VISIBLE_DEVICES= 0
CUDA_VISIBLE_DEVICES= 4
CUDA_VISIBLE_DEVICES= 2
CUDA_VISIBLE_DEVICES= 7
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
n_models: 1
capt

Processing in parallel:   5%|▌         | 1/20 [00:31<10:05, 31.85s/it]

capturing graph
capturing graph
n_models: 1
capturing graph
capturing graph
n_models: 1
capturing graph
n_models: 1
capturing graph
n_models: 1
n_models: 1
n_models: 1


Processing in parallel: 100%|██████████| 20/20 [01:06<00:00,  3.32s/it]


In [1]:
model.models[1].run_in_parallel = False
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    #model.infer(this_data_out[:2])
    res['time_taken'].append((time.time()-t)/len(this_data_out[:2])/60)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    print(d.family, res_all)
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

NameError: name 'model' is not defined